# MRIO seminal works

#### Objectives
- Learn how to replicate the methods and results from a few studies. 
- Understand and explain main results from EEIOA studies using MRIO seminal works
- Perform an MRIO analysis in Python
- Interpret the results using the MRIO seminal


### Part 1: Understanding and explaining MRIO results
Wood et al. (2018)  explore the way in which international trade contributes to 4 environmental pressures from the consumption of a population. Among their results, they show the growth in consumption-based footprints per capita between 1995 and 2011 for 11 world regions:

|![Figure 1](figures/growth_footprint_pc.png)|
|:--:|
|Figure 1. Growth in consumption-based footprints per capita between 1995 and 2011 for 11 world regions (1995 = 1) retrieved from Wood et al. (2018). GHG = greenhouse gas; OECD = Organization for Economic Cooperation and Development|


##### Questions 1
Does figure 1 present any aspects related to MRIO seminal (e.g., logic, value visualization, language, etc.)? Provide two examples.


In [3]:
import ipywidgets as widgets

a_1 = widgets.Textarea(placeholder="Add here answer to question 1", description="Answer 1", disabled=False, layout=widgets.Layout(width='1000px', height="100px"))

display(a_1)

Textarea(value='', description='Answer 1', layout=Layout(height='100px', width='1000px'), placeholder='Add her…

##### Question 2

Which environmental footprints increase or decrease in Europe between 1995 and 2011?

In [5]:
a_2 = widgets.Textarea(placeholder="Add here answer to question 2", description="Answer 1", disabled=False, layout=widgets.Layout(width='1000px', height="100px"))

display(a_2)

Textarea(value='', description='Answer 1', layout=Layout(height='100px', width='1000px'), placeholder='Add her…

##### Question 3
Describe step-by-step how would you develop figure, bringing the possible equations required. Note: To facilitate the process, you can focus on one environmental pressure in one region, for one specific year (e.g., GHG emissions footprint in Europe for 2011). 

In [6]:
a_3 = widgets.Textarea(placeholder="Add here answer to question 3", description="Answer 3", disabled=False, layout=widgets.Layout(width='1000px', height="100px"))

display(a_3)

Textarea(value='', description='Answer 3', layout=Layout(height='100px', width='1000px'), placeholder='Add her…

### Part 2: Python exercise
Hertwich and Peters (2009) developed an analysis of the carbon footprint of multiple countries in 2001. Their results show the carbon footprint of different final demand categories per product.

In this exercise you will reproduce these results using EXIOBASE and compare any changes between 2001 and 2011

|![Figure 2](figures/global_CO2_footprint.png)|
|:--:|
|Figure 2. Global CO2 footprint for different consumption categories and users retrieved from Hertwich and Peters (2009)|

##### Steps in this exercise:
- Import data
- Calculate Leontief inverse and environmental intesity matrix
- Final demand contribution to environmental footprint

### Instructions
- Save and unzip the data files to the current working directory
- First read (tab delimited) data as DataFrame, which is a data strcuture in the Pandas library, 
- Convert the DataFrame to numpy array. To learn more about DataFrame and read_csv [click here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html.)

### Import modules and data

In [ ]:
# Import modules
import pandas as pd 
import numpy as np

In [ ]:
# Import EXIOBASE
path = 'IOT_2011_pxp'    # add name of folder where data is stored                                
A = pd.read_csv(path + '/A.txt', sep='\t',
                index_col=[0, 1], header=[0, 1])  # A matrix
Y = pd.read_csv(path + '/Y.txt', sep='\t',
                index_col=[0, 1], header=[0, 1])  # y matrix
F = pd.read_csv(path + '/satellite/F.txt', sep='\t',
                index_col=[0], header=[0, 1])  # satellite matrix
F_hh = pd.read_csv(path + '/satellite/F_hh.txt', sep='\t',
                index_col=[0], header=[0, 1])  # satellite for FD matrix

### Calculate the Leontief inverse

In [27]:
## Leontief inverse matrix
I = np.identity(len(A))           # An identy matrix of the size as A-matrix
L = np.linalg.inv(I - A)

NameError: name 'A' is not defined

In [ ]:
## Calculate total output vector (x)

y_total = Y.sum(1)      # Sum of all final demand categories
x = L @ y_total         # Total outputs

### Calculate environmental intensity matrix
From satellite matrix (F), select row for CO2 (i.e., F_co2). This can be done using pd.loc

In [ ]:
## Environmental intensities

co2_lab = 'CO2 - combustion - air' # Label
F_co2 = F.loc[co2_lab, :]  # select CO2 vector
f_co2 = F_co2/x.transpose() # co2 intensity vector
f_co2 = f_co2.replace([np.inf, -np.inf], np.nan).replace(np.nan, 0)  # replacing inf, -inf, and nan with zeros

### Contribution of Final Demand to the CO2 footprint
From the final demand matrix (Y), calculate the global final demand of households (y_hh), government expenditures (y_gov) and gross capital formation (y_cap).

In [ ]:
## Calculation y_hh, y_gov, and y_cap
hh_lab = 'Final consumption expenditure by households' 
gov_lab = 'Final consumption expenditure by government'	
cap_lab = 'Gross fixed capital formation'

y_hh = Y.xs(hh_lab, axis=1, level=1, drop_level=False)  # selecting hh columns for all countries/regions
y_hh = y_hh.sum(1)  # global final expenditure by household 

y_gov = Y.xs(gov_lab, axis=1, level=1, drop_level=False)  # selecting gov columns for all countries/regions
y_gov = y_gov.sum(1)  # global final expenditure by household 

y_cap = Y.xs(cap_lab, axis=1, level=1, drop_level=False)  # selecting cap columns for all countries/regions
y_cap = y_cap.sum(1)  # global capital formation 

### Calculation CO2 footprint per final demand category per product

In [ ]:
co2_hh = f_co2 @ L @ np.diag(y_hh) # CO2 footprint by household expenditures
co2_gov = f_co2 @ L @ np.diag(y_gov) # CO2 footprint by goverment expenditures 
co2_cap = f_co2 @ L @ np.diag(y_cap) # CO2 footprint by capital formation 

At this point, it should be 3 vectors (for each final demand category from point 2) of 1 row with 9800 columns (i.e., 49 regions x 200 products)

Now reshape each vector using np.reshape so that you have 3 matrices with 49 rows (for each country/region) and 200 columns (for each product)

In [ ]:
## Reshape vectors

co2_hh = np.array(co2_hh).reshape(49,200) # reshape to 49 countries/regions and 200 products
co2_hh_sum = np.sum(co2_hh, axis=0) # sum per product category

co2_gov = np.array(co2_gov).reshape(49,200) # reshape to 49 countries/regions and 200 products
co2_gov_sum = np.sum(co2_gov, axis=0) # sum per product category

co2_cap = np.array(co2_cap).reshape(49,200) # reshape to 49 countries/regions and 200 products
co2_cap_sum = np.sum(co2_cap, axis=0) # sum per product category

## Build new pandas dataframe

df = pd.DataFrame([co2_hh_sum, co2_gov_sum, co2_cap_sum])

### Separate results by sectors

- Extraction
- Manufacturing
- Construction
- Services

In [ ]:
### Index product categories

ext_ind = list(range(0,41)) # extraction categories (including agriculture, and mining)
man_ind = list(range(41,149)) # manufacturing (including food, clothing, other products)
con_ind = list(range(149,151)) # construction categories
ser_ind = list(range(151,200))  # services categories

## Sum per product group

ext = df.iloc[:, ext_ind].sum(1)
man = df.iloc[:, man_ind].sum(1)
con = df.iloc[:, con_ind].sum(1)
ser = df.iloc[:, ser_ind].sum(1)

In [ ]:
## Re-group dataframe and add labels

df_new = pd.concat([ext, man, con, ser], axis=1)
df_new.index = ['Households', 'Goverment', 'Investment']
df_new.columns = ['Agriculture and Mining', 'Manufacturing', 'Construction', 'Services']

### Visualize the results
Create a bar graph showing the final contribution per product category to the CO2 footprint

Note: This can be done by using:
df_new.plot.barh(stacked=True)

In [ ]:
## Create a vertical bar chart

ef_name = 'CO2 footprint (tonnes)'
fig = df_new.plot.barh(stacked=True)
fig.set_xlabel("CO2 footprint (kg)")
fig.set_ylabel(ef_name)
fig.set_title(ef_name + " per final demand category")
fig.legend(loc='center left', bbox_to_anchor=(1, 0.5))

##### Question 4

Which sector has a major contribution to the CO2 footprint of each final demand category?

In [29]:
a_4 = widgets.Textarea(placeholder="Add here answer to question 4", description="Answer 4", disabled=False, layout=widget.Layout(width='1000px', height="100px"))

display(a_4)

Textarea(value='', description='Answer 4', layout=Layout(height='100px', width='1000px'), placeholder='Add her…

##### Question 5

What are the differences between the 2001 and 2011?

In [31]:
a_5 = widgets.Textarea(placeholder="Add here answer to question 5", description="Answer 5", disabled=False, layout=widget.Layout(width='1000px', height="100px"))

display(a_5)

Textarea(value='', description='Answer 5', layout=Layout(height='100px', width='1000px'), placeholder='Add her…

## References
Hertwich, E. G., & Peters, G. P. (2009). Carbon footprint of nations: A global, trade-linked analysis. Environmental Science and Technology, 43(16), 6414–6420. https://doi.org/10.1021/es803496a

Wood, R., Stadler, K., Simas, M., Bulavskaya, T., Giljum, S., Lutter, S., & Tukker, A. (2018). Growth in Environmental Footprints and Environmental Impacts Embodied in Trade: Resource Efficiency Indicators from EXIOBASE3. Journal of Industrial Ecology, 22(3), 553–562. https://doi.org/10.1111/jiec.12735